# Task 2
## 1. Env set up
1. ... (already done before)
2. pip install scikit-learn(including sklearn, scipy)
---
## 2. Learning/Working route
1. figure out task-involved knowledge range, in this case package calling
2. watch some short videos about these package to quickly get knowing what they do, have, and their advantages
3. check their usage in the official documents
4. learn new concepts like OneHot encoding, ask AI about its common realizations, and learn related methods
5. ask AI about common ways of processing numerical data and do it under guidance
6. when visualizing, using AI to help adjust the parameters
---
## 3. Work Sequentially
#### a. Before start

In [3]:
import numpy as np
import scipy as sp
import sklearn as sk

train_data = np.read_csv('../MBAAdmission/train.csv')
test_data = np.read_csv('../MBAAdmission/test.csv')


AttributeError: module 'numpy' has no attribute 'read_csv'